# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: Low Rank Adaptation
* Model: distilbert-base-uncased
* Evaluation approach: accuracy
* Fine-tuning dataset: sms_spam

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
! pip install -q "datasets==2.15.0"

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
from datasets import load_dataset
dataset = load_dataset("sms_spam", split="train").train_test_split(test_size=0.2, shuffle=True, seed=23)

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 5574/5574 [00:00<00:00, 31122.19 examples/s]


In [2]:
dataset["train"]

Dataset({
    features: ['sms', 'label'],
    num_rows: 4459
})

In [3]:
dataset["test"]

Dataset({
    features: ['sms', 'label'],
    num_rows: 1115
})

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
splits = ["train","test"]

tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
    lambda x: tokenizer(x["sms"], truncation=True), batched=True)

tokenizer_config.json: 100%|██████████| 28.0/28.0 [00:00<00:00, 130kB/s]
config.json: 100%|██████████| 483/483 [00:00<00:00, 1.94MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 4.06MB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 9.08MB/s]
Map: 100%|██████████| 1115/1115 [00:00<00:00, 6074.51 examples/s]


In [5]:
tokenized_dataset["train"]

Dataset({
    features: ['sms', 'label', 'input_ids', 'attention_mask'],
    num_rows: 4459
})

In [6]:
tokenized_dataset["test"]

Dataset({
    features: ['sms', 'label', 'input_ids', 'attention_mask'],
    num_rows: 1115
})

In [7]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label={0: "not spam", 1: "spam"},
    label2id={"not spam": 0, "spam": 1},
)

for param in model.parameters():
    param.requires_grad = True

model.safetensors: 100%|██████████| 268M/268M [00:01<00:00, 200MB/s]  
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/spam_not_spam",
        learning_rate=1e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=1,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.059137,0.984753


TrainOutput(global_step=279, training_loss=0.11283616096742692, metrics={'train_runtime': 33.7946, 'train_samples_per_second': 131.944, 'train_steps_per_second': 8.256, 'total_flos': 71170697181816.0, 'train_loss': 0.11283616096742692, 'epoch': 1.0})

In [9]:
trainer.evaluate()

{'eval_loss': 0.059137120842933655,
 'eval_accuracy': 0.9847533632286996,
 'eval_runtime': 2.1464,
 'eval_samples_per_second': 519.48,
 'eval_steps_per_second': 32.613,
 'epoch': 1.0}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [11]:
print(model)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [14]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r = 16,
    lora_alpha=32,
    target_modules=["q_lin", "k_lin", "v_lin"],
    lora_dropout=0.05,
    bias="none"
)

lora_model = get_peft_model(model, config)
lora_model.print_trainable_parameters()

trainable params: 442,368 || all params: 67,397,378 || trainable%: 0.6563578778984548


In [15]:
lora_model.save_pretrained("distilbert-base-uncased-lora")

In [16]:
lora_dataset = load_dataset("sms_spam", split="train").train_test_split(test_size=0.1, shuffle=True, seed=567)
lora_tokenized_dataset = {}
for split in splits:
    lora_tokenized_dataset[split] = lora_dataset[split].map(
    lambda x: tokenizer(x["sms"], truncation=True), batched=True)

Map: 100%|██████████| 558/558 [00:00<00:00, 8173.72 examples/s]


In [17]:
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/lora_spam_not_spam",
        learning_rate=1e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=1,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=lora_tokenized_dataset["train"],
    eval_dataset=lora_tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.044919,0.991039


TrainOutput(global_step=314, training_loss=0.041003728368479736, metrics={'train_runtime': 23.6964, 'train_samples_per_second': 211.678, 'train_steps_per_second': 13.251, 'total_flos': 82838512601088.0, 'train_loss': 0.041003728368479736, 'epoch': 1.0})

In [18]:
lora_model.save_pretrained("distilbert-base-uncased-lora-1epoch")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [19]:
from peft import AutoPeftModelForSequenceClassification
lora_model = AutoPeftModelForSequenceClassification.from_pretrained("distilbert-base-uncased-lora-1epoch")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/lora_spam_not_spam",
        learning_rate=1e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=1,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=lora_tokenized_dataset["train"],
    eval_dataset=lora_tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

In [21]:
trainer.evaluate()

{'eval_loss': 0.044918518513441086,
 'eval_accuracy': 0.9910394265232975,
 'eval_runtime': 1.2834,
 'eval_samples_per_second': 434.783,
 'eval_steps_per_second': 27.271}

### The previous model has an accuracy of 0.9847, the fine tuned model has an accuracy of 0.9910.